In [10]:
# Загружаем библиотеки
from pandas import Series
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup  
import requests 

import re
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [6]:
# Загрузим наши тестовые данные
test = pd.read_csv('test.csv')

## Анализ тестовых данных

In [3]:
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [4]:
# Посмотрим на пропуски в тестовой выборке
test.isnull().sum()

bodyType                    0
brand                       0
car_url                     0
color                       0
complectation_dict      28268
description                 0
engineDisplacement          0
enginePower                 0
equipment_dict           9996
fuelType                    0
image                       0
mileage                     0
modelDate                   0
model_info                  0
model_name                  0
name                        0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы                   0
Владение                22691
ПТС                         1
Привод                      0
Руль                        0
Состояние                   0
Таможня                     0
dtype: int64

Удалим бесполезную и дублирующуюся информацию, а также удалим столбцы, в которых значительное количество пропусков.

In [5]:

for_drop = ['car_url', 'complectation_dict', 'equipment_dict', 'image', 'Владение', 'parsing_unixtime', 'super_gen', 
            'sell_id', 'priceCurrency', 'model_info', 'description','name', 'vendor', 'ПТС', 'Таможня']
test.drop(for_drop, axis=1, inplace=True)


In [6]:
test

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,Владельцы,Привод,Руль,Состояние
0,лифтбек,SKODA,синий,1.2 LTR,105 N12,бензин,74000,2013,OCTAVIA,5,2014,LIFTBACK ROBOT 1.2,роботизированная,3 или более,передний,Левый,Не требует ремонта
1,лифтбек,SKODA,чёрный,1.6 LTR,110 N12,бензин,60563,2017,OCTAVIA,5,2017,LIFTBACK MECHANICAL 1.6,механическая,1 владелец,передний,Левый,Не требует ремонта
2,лифтбек,SKODA,серый,1.8 LTR,152 N12,бензин,88000,2013,SUPERB,5,2014,LIFTBACK ROBOT 1.8,роботизированная,1 владелец,передний,Левый,Не требует ремонта
3,лифтбек,SKODA,коричневый,1.6 LTR,110 N12,бензин,95000,2013,OCTAVIA,5,2014,LIFTBACK AUTOMATIC 1.6,автоматическая,1 владелец,передний,Левый,Не требует ремонта
4,лифтбек,SKODA,белый,1.8 LTR,152 N12,бензин,58536,2008,OCTAVIA,5,2012,LIFTBACK AUTOMATIC 1.8,автоматическая,1 владелец,передний,Левый,Не требует ремонта
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34681,седан,BMW,коричневый,1.6 LTR,136 N12,бензин,115000,2011,3ER,4,2014,SEDAN AUTOMATIC 1.6,автоматическая,3 или более,задний,Левый,Не требует ремонта
34682,седан,BMW,чёрный,2.0 LTR,190 N12,дизель,98000,2016,5ER,4,2018,SEDAN AUTOMATIC 2.0,автоматическая,1 владелец,полный,Левый,Не требует ремонта
34683,седан,BMW,серый,2.5 LTR,170 N12,бензин,360000,1995,5ER,4,1997,SEDAN AUTOMATIC 2.5,автоматическая,3 или более,задний,Левый,Не требует ремонта
34684,внедорожник 5 дв.,BMW,коричневый,2.0 LTR,184 N12,дизель,90500,2012,X1,5,2013,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,2 владельца,полный,Левый,Не требует ремонта


Осталось 19 признаков, далее займемся парсингом данных признаков.

In [7]:
# Извлечем марки машин которые нужно спарсить
brands = list(test.brand.unique())

# Выберем названия столбцов
columns = list(test.columns)

Выполним парсинг ссылок на объявления, затем выполним парсинг признаков с данных ссылок. Парсинг ссылок будем выполнять по маркам автомобилей имеющихся в тестовой выборке. Создадим два словаря с данными, из которого будем уже составлять наш трейновый датасет.

In [8]:
# Собственно парсинг ссылок
# Создадим словарь для данных с колонками 
data_dict = {}
for column in test.columns:
    data_dict[column] = []
data_dict['price'] = []

url = 'https://auto.ru/moskva/cars/'
for car_brand in brands:
    brand_url = url + car_brand + '/used/?page='
    print(car_brand)
    for number in range(1, 5):
        
        response = requests.get(brand_url+str(number), headers={'User-Agent': 'Mozilla/5.0'})
        page = BeautifulSoup(response.text, 'html.parser')
        link_list = page.find_all('a', class_='Link ListingItemTitle-module__link')
        if len(link_list) !=0:
            for link in link_list:
                response = requests.get(link['href'])
                if response.status_code != 200:
                    raise BaseException("response code" + str(response.status_code))
                
                response.encoding = 'utf-8'
                page = BeautifulSoup(response.text, 'html.parser')
                first_dict = ast.literal_eval(str(page.find_all('script', attrs = {'type':'application/ld+json'})[0])[35:-9])
              
                data_dict['brand'].append(first_dict['brand'])
                data_dict['bodyType'].append(first_dict['bodyType'])
                data_dict['color'].append(first_dict['color'])
                data_dict['engineDisplacement'].append(first_dict['vehicleEngine']['engineDisplacement'])
                data_dict['enginePower'].append(first_dict['vehicleEngine']['enginePower'])
                data_dict['fuelType'].append(first_dict['fuelType'])
                data_dict['modelDate'].append(first_dict['modelDate'])
                try: 
                    data_dict['model_name'].append(first_dict['name'].split(' ')[1])
                except KeyError:
                    data_dict['model_name'].append(np.nan)
                data_dict['numberOfDoors'].append(first_dict['numberOfDoors'])
                data_dict['productionDate'].append(first_dict['productionDate'])
                data_dict['vehicleConfiguration'].append(first_dict['vehicleConfiguration'])
                data_dict['vehicleTransmission'].append(first_dict['vehicleTransmission'])
                           
                data_dict['mileage'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_kmAge")[0].text.replace('\xa0', '')[6:-2])
                try:
                    data_dict['Владельцы'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_ownersCount")[0].text.replace('\xa0', '')[9:-9])
                except:
                    data_dict['Владельцы'].append(np.nan)

                data_dict['Привод'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_drive")[0].text[6:])
                data_dict['Руль'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_wheel")[0].text[4:])
                data_dict['Состояние'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_state")[0].text[9:])
                try:
                    data_dict['price'].append(first_dict['offers']['price'])
                except KeyError:
                    data_dict['price'].append(np.nan)

        else:
            break


SKODA
AUDI
HONDA
VOLVO
BMW
NISSAN
INFINITI
MERCEDES
TOYOTA
LEXUS
VOLKSWAGEN
MITSUBISHI


In [9]:
data_dict_1_4=data_dict

In [10]:
#data_dict_1_5=data_dict

In [ ]:
with open('data1.json', 'w') as fp:
    json.dump(data_dict_1_5, fp)

In [10]:
# Собственно парсинг ссылок
# Создадим словарь для данных с колонками 
data_dict = {}
for column in test.columns:
    data_dict[column] = []
data_dict['price'] = []

url = 'https://auto.ru/moskva/cars/'
for car_brand in brands:
    brand_url = url + car_brand + '/used/?page='
    print(car_brand)
    for number in range(5, 11):
        
        response = requests.get(brand_url+str(number), headers={'User-Agent': 'Mozilla/5.0'})
        page = BeautifulSoup(response.text, 'html.parser')
        link_list = page.find_all('a', class_='Link ListingItemTitle-module__link')
        if len(link_list) !=0:
            for link in link_list:
                response = requests.get(link['href'])
                if response.status_code != 200:
                    raise BaseException("response code" + str(response.status_code))
               
                response.encoding = 'utf-8'
                page = BeautifulSoup(response.text, 'html.parser')
                first_dict = ast.literal_eval(str(page.find_all('script', attrs = {'type':'application/ld+json'})[0])[35:-9])
              
                data_dict['brand'].append(first_dict['brand'])
                data_dict['bodyType'].append(first_dict['bodyType'])
                data_dict['color'].append(first_dict['color'])
                data_dict['engineDisplacement'].append(first_dict['vehicleEngine']['engineDisplacement'])
                data_dict['enginePower'].append(first_dict['vehicleEngine']['enginePower'])
                data_dict['fuelType'].append(first_dict['fuelType'])
                data_dict['modelDate'].append(first_dict['modelDate'])
                try: 
                    data_dict['model_name'].append(first_dict['name'].split(' ')[1])
                except KeyError:
                    data_dict['model_name'].append(np.nan)
                data_dict['numberOfDoors'].append(first_dict['numberOfDoors'])
                data_dict['productionDate'].append(first_dict['productionDate'])
                data_dict['vehicleConfiguration'].append(first_dict['vehicleConfiguration'])
                data_dict['vehicleTransmission'].append(first_dict['vehicleTransmission'])
                           
                data_dict['mileage'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_kmAge")[0].text.replace('\xa0', '')[6:-2])
                try:
                    data_dict['Владельцы'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_ownersCount")[0].text.replace('\xa0', '')[9:-9])
                except:
                    data_dict['Владельцы'].append(np.nan)

                data_dict['Привод'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_drive")[0].text[6:])
                data_dict['Руль'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_wheel")[0].text[4:])
                data_dict['Состояние'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_state")[0].text[9:])
                try:
                    data_dict['price'].append(first_dict['offers']['price'])
                except KeyError:
                    data_dict['price'].append(np.nan)

        else:
            break

SKODA
AUDI
HONDA
VOLVO
BMW


ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [ ]:
page

In [13]:
for i in data_dict['bodyType']:
    print(i)

внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
лифтбек
лифтбек
внедорожник 5 дв.
внедорожник 5 дв.
универсал 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
лифтбек
лифтбек
лифтбек
внедорожник 5 дв.
лифтбек
хэтчбек 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
лифтбек
хэтчбек 5 дв.
универсал 5 дв.
лифтбек
универсал 5 дв.
внедорожник 5 дв.
внедорожник 5 дв.
лифтбек
лифтбек
лифтбек
хэтчбек 5 дв.
лифтбек
лифтбек
лифтбек
лифтбек
лифтбек
лифтбек
лифтбек
седан
внедорожник 5 дв.
универсал 5 дв.
внедорожник 5 дв.
лифтбек
лифтбек
лифтбек
лифтбек
седан
внедорожник 5 дв.
внедорожник 5 дв.
лифтбек
лифтбек
лифтбек
внедорожник 5 дв.
лифтбек
хэтчбек 5 дв.
лифтбек
хэтчбек 5 дв.
лифтбек
лифтбек
лифтбек
внедорожник 5 дв.
лифтбек
лифтбек
внедорожник 5 дв.
лифтбек
лифтбек
лифтбек
лифтбек
хэтчбек 5 дв.
лифтбек
лифтбек
хэтчбек 5 дв.
компактвэн
внедорожник 5 дв.
лифтбек

In [13]:
for i in range(1, 5):
    print(i)

1
2
3
4


In [48]:
url = 'https://auto.ru/moskva/cars/used/'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
page = BeautifulSoup(response.text, 'html.parser')
link_list = page.find_all('a', class_='Link ListingItemTitle-module__link')

In [49]:
url_1 = link_list[0]['href']
response = requests.get(url_1)
response.encoding = 'utf-8'
page = BeautifulSoup(response.text, 'html.parser')
first_dict = ast.literal_eval(str(page.find_all('script', attrs = {'type':'application/ld+json'})[0])[35:-9])
second_dict = [info.text for info in page.find_all('span', class_="CardInfoRow__cell")]


In [33]:
data_dict = {}
for column in test.columns:
    data_dict[column] = []
data_dict['price'] = []

In [6]:
import pandas as pd

In [1]:
import json  
with open('data.json') as fp:
   data_dict = json.load(fp)



In [10]:
data_1 = pd.DataFrame(data_dict)[:50000]

In [13]:
data_2 = pd.DataFrame(data_dict)[50000:]

In [16]:
data_1.to_csv('data_1.csv', index = False)


In [17]:
data_2.to_csv('data_2.csv', index = False)

In [4]:
len(data_dict['car_url'])

94752

In [3]:
len(data_dict['price'])

94752

In [15]:
data_dict['modelName'] = data_dict.pop('model_name')

In [19]:
with open('data_full.json', 'w') as fp:
    json.dump(data_dict, fp)

In [ ]:
url = 'https://auto.ru/moskva/cars/'
for car_brand in brands:
    brand_url = url + car_brand + '/used/?page='
    cars_urls[car_brand] = []
    print(car_brand)
    for number in range(1, 100):
        response = requests.get(brand_url+str(number), headers={'User-Agent': 'Mozilla/5.0'})
        page = BeautifulSoup(response.text, 'html.parser')
        link_list = page.find_all('a', class_='Link ListingItemTitle-module__link')
        if len(link_list) !=0:
            for link in link_list:
                cars_urls[car_brand].append(link['href'])
        else:
            break
with open('data.json', 'w') as fp:
    json.dump(cars_urls, fp)

In [12]:
for i in range(1,len(data_dict['car_url']),100):
    for url in data_dict['car_url'][i: i+100]:
        response = requests.get(url)
        if response.status_code != 200:
            print('Ожидаю 0.5 секунды...')  
            time.sleep(0.5)
            response = requests.get(url)
            
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
        first_dict = ast.literal_eval(str(page.find_all('script', attrs = {'type':'application/ld+json'})[0])[35:-9])
              
        try:
            data_dict['brand'].append(first_dict['brand'])
        except:
            data_dict['brand'].append(np.nan)
        try:
            data_dict['bodyType'].append(first_dict['bodyType'])
        except:
            data_dict['bodyType'].append(np.nan)
        try:
            data_dict['color'].append(first_dict['color'])
        except:
            data_dict['color'].append(np.nan)
        try:
            data_dict['engineDisplacement'].append(first_dict['vehicleEngine']['engineDisplacement'])
        except:
            data_dict['engineDisplacement'].append(np.nan)
        try:
            data_dict['enginePower'].append(first_dict['vehicleEngine']['enginePower'])
        except:
            data_dict['enginePower'].append(np.nan)
        try:
            data_dict['fuelType'].append(first_dict['fuelType'])
        except:
            data_dict['fuelType'].append(np.nan)
        try:
            data_dict['modelDate'].append(first_dict['modelDate'])
        except:
            data_dict['modelDate'].append(np.nan)
        try: 
            data_dict['model_name'].append(first_dict['name'].split(' ')[1])
        except KeyError:
            data_dict['model_name'].append(np.nan)
        try:
            data_dict['numberOfDoors'].append(first_dict['numberOfDoors'])
        except:
            data_dict['numberOfDoors'].append(np.nan)
        try:
            data_dict['productionDate'].append(first_dict['productionDate'])
        except:
            data_dict['productionDate'].append(np.nan)    
        try:
            data_dict['vehicleConfiguration'].append(first_dict['vehicleConfiguration'])
        except:
            data_dict['vehicleConfiguration'].append(np.nan)
        try:
            data_dict['vehicleTransmission'].append(first_dict['vehicleTransmission'])  
        except:
            data_dict['vehicleTransmission'].append(np.nan)
        try:
            data_dict['mileage'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_kmAge")[0].text.replace('\xa0', '')[6:-2])
        except:
            data_dict['mileage'].append(np.nan)
        try:
            data_dict['Владельцы'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_ownersCount")[0].text.replace('\xa0', '')[9:-9])
        except:
            data_dict['Владельцы'].append(np.nan)
        try:
            data_dict['Привод'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_drive")[0].text[6:])
        except:
            data_dict['Привод'].append(np.nan)
        try:
            data_dict['Руль'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_wheel")[0].text[4:])
        except:
            data_dict['Руль'].append(np.nan)
        try:
            data_dict['Состояние'].append(page.find_all('li', class_="CardInfoRow CardInfoRow_state")[0].text[9:])
        except:
            data_dict['Состояние'].append(np.nan)
        try:
            data_dict['price'].append(first_dict['offers']['price'])
        except:
            data_dict['price'].append(np.nan)
    with open('data.json', 'w') as fp:
        json.dump(data_dict, fp)
    print('+100')  



+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
Ожидаю 0.5 секунды...
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
Ожидаю 0.5 секунды...
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
Ожидаю 0.5 секунды...
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
Ожидаю 0.5 секунды...
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+1

In [9]:
for i in range(1,len(data_dict['car_url']),100):
    for url in data_dict['car_url'][i: i+100]:
        response = requests.get(url)
        if response.status_code != 200:
            print('Ожидаю 0.5 секунды...')  
            time.sleep(0.5)
            response = requests.get(url)
            
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
        first_dict = ast.literal_eval(str(page.find_all('script', attrs = {'type':'application/ld+json'})[0])[35:-9])
              
        try: 
            data_dict['model_name'].append(first_dict['name'].split(' ')[1])
        except KeyError:
            data_dict['model_name'].append(np.nan)

    with open('data.json', 'w') as fp:
        json.dump(data_dict, fp)
    print('+100')  

+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
Ожидаю 0.5 секунды...
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
Ожидаю 0.5 секунды...
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+100
+

In [132]:
len(data_dict['brand'])

4109

In [133]:
len(data_dict['price'])

4109

In [11]:
len(data_dict['model_name'])

34375

In [17]:
data_dict.keys()

dict_keys(['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'mileage', 'modelDate', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'Владельцы', 'Привод', 'Руль', 'Состояние', 'price', 'model_name'])